In [1]:
import polars as pl
import pandas as pd
import os
import json
import numpy as np
from catboost import CatBoostClassifier, CatBoostRanker, Pool, sum_models
import gc
from ebrec.evaluation.metrics_protocols import *
from fastauc.fastauc.fast_auc import CppAuc
import argparse
import joblib


In [2]:
N_BATCH = 10
RANKER = True
model_path = '/home/ubuntu/experiments/catboost_rnk_recsys_train_val'
dataset_path = '/home/ubuntu/experiments/preprocessing_train_new_with_recsys'
catboost_params = {
    "iterations": 2421,
    "learning_rate": 0.061372161824290145,
    "rsm": 0.681769606695633,
    "reg_lambda": 0.4953354255208565,
    "grow_policy": "SymmetricTree",
    "bootstrap_type": "MVS",
    "subsample": 0.5108219602277233,
    "random_strength": 14.089062269780399,
    "fold_permutation_block": 39,
    "border_count": 34,
    "sampling_frequency": "PerTreeLevel",
    "score_function": "Cosine",
    "depth": 8,
    "mvs_reg": 0.0015341832942953422
}


In [3]:
with open(os.path.join(dataset_path, 'data_info.json')) as data_info_file:
        data_info = json.load(data_info_file)
        
models = []   
for batch in range(N_BATCH):
            if RANKER:
                model = CatBoostRanker(
                    **catboost_params, cat_features=data_info['categorical_columns'])
            else:
                model = CatBoostClassifier(**catboost_params, cat_features=data_info['categorical_columns'])
                
            model.load_model(model_path + f'/models/model_{batch}.cbm', format='cbm')
            models.append(model)

weights = [1/N_BATCH] * N_BATCH
model = sum_models(models, weights=weights,ctr_merge_policy='IntersectingCountersAverage')
joblib.dump(model, os.path.join(model_path, 'model.joblib'))

['/home/ubuntu/experiments/catboost_rnk_recsys_train_val/model.joblib']